## RMSProp combination with mini batch then stochastic 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

#### data prepare

In [3]:
data = np.loadtxt("MultipleLR.csv", delimiter=",")
X = data[:, 0:3]
ones = np.ones(len(X))
X = np.insert(X, 0, ones, axis=1)
y = data[:, -1]

#### RMSProp in mini batch

In [4]:
def RMSProp_mini_batch(X, y, alpha, beta, epsilon, iterations, batch_size):
    m, n = X.shape
    thetas = np.zeros((n, 1))
    v = 0
    J = []
    old_cost = 0
    counter = 0
    for _ in range(iterations):
        for i in range(0, m, batch_size):

            x_batched = X[i : i + batch_size]
            y_batched = y[i : i + batch_size]

            hyp = np.dot(x_batched, thetas)
            
            error = hyp - y_batched.reshape(len(y_batched), 1)
            cost = np.sum((error)**2) / (2*len(y_batched))
            J.append(cost)
            gradient = np.sum(x_batched.T @ error) / len(y_batched)

            v = (beta * v) + ((1 - beta) * gradient**2)
            thetas = thetas - ((alpha * gradient) / (np.sqrt(v) + epsilon))

        if abs(old_cost - cost) < 1e-3:
            break
        old_cost = cost
        counter += 1
    y_predicted = X @ thetas
    return J, counter, y_predicted

In [9]:
J, counter, y_predicted = RMSProp_mini_batch(X, y, 0.01, 0.3, 1e-8, 1000, 5)
print(f"epochs = {counter}")
print(f"r2_score = {r2_score(y_predicted, y)}")

epochs = 16
r2_score = 0.9708987417613048


### RMSProp in stchastic

In [6]:
def RMSProp_stochastic(X, y, alpha, beta, epsilon, iterations):
    m, n = X.shape
    thetas = np.zeros((n, 1))
    v = 0
    J = []
    old_cost = 0
    counter = 0
    for _ in range(iterations):
        for i in range(0, m):
            hyp = np.dot(X[i].reshape(1, n), thetas)
            
            error = hyp - y[i]
            cost = error**2
            J.append(cost)
            
            gradient = (X[i].reshape(1, n).T @ error) * 2

            v = (beta * v) + ((1 - beta) * gradient**2)
            thetas = thetas - ((alpha * gradient) / (np.sqrt(v) + epsilon))
            
        if abs(old_cost - cost) < 1e-3:
            break
        old_cost = cost
        counter += 1
    y_predicted = X @ thetas
    return J, counter, y_predicted

In [8]:
J, counter, y_predicted = RMSProp_stochastic(X, y, 0.01, 0.999, 1e-8, 1000)
print(f"epochs = {counter}")
print(f"r2_score = {r2_score(y_predicted, y)}")

epochs = 1
r2_score = 0.9780529187065564
